In [1]:
%run EconIndicators.ipynb

In [2]:
rename_dict = {
    'FRED/DEXUSAL':'AUS',
    'FRED/DEXBZUS':'BRA',
    'FRED/DEXUSUK':'GBR',
    'FRED/DEXCAUS':'CAD',
    'FRED/DEXCHUS':'CHN',
    'FRED/DEXUSEU':'EUU',
    'FRED/DEXINUS':'IND',
    'FRED/DEXJPUS':'JPN',
    'FRED/DEXMAUS':'MYS', 
    'FRED/DEXMXUS':'MEX',
    'FRED/DEXUSNZ':'NZL', 
    'FRED/DEXNOUS':'NOR',
    'FRED/DEXSIUS':'SGP',
    'FRED/DEXSFUS':'ZAF',
    'FRED/DEXKOUS':'KOR',
    'FRED/DEXSDUS':'SWE',
    'FRED/DEXSZUS':'CHE',
    'FRED/DEXTHUS':'THB'
}

In [3]:
# Getting Training Economic Indicators from World Bank (From EconIndicators.ipynb)
raw_data = getAllIndicatorsTraining(20, full_country_basket)
norm_data = relativeStrength(raw_data)

Retrying API call for: GDP in 10 seconds.
Retrying API call for: UEM in 10 seconds.


In [4]:
# Reading in annualized volatility data for training
avol_csv = pd.read_csv("annualized_volatility.csv", index_col = "Date")
avol_csv = avol_csv.loc[2000:]
avol_csv.rename(columns = rename_dict, inplace = True)
avol_csv.drop(columns = ['FRED/DEXDNUS', 'FRED/DEXHKUS', 'FRED/DEXTAUS', 'FRED/DEXVZUS'], inplace = True)
#avol_csv.head()

In [5]:
# Creating df to match the format of the df provided by EconIndicators
avol_dict = {}
for country in avol_csv.columns:
    for year in avol_csv[country].index:
        # Subtracting a year as the previous year's volatility will be predicting the next 
        avol_dict[country + str(year-1)] = avol_csv[country][year]
        
avol_df = pd.DataFrame.from_dict(avol_dict, orient = "index", columns = ["Annual Volatility"])
#avol_df.head()

In [6]:
features_df = pd.concat([norm_data, avol_df], axis = "columns", join = "inner")
features_df.head()

,Year,GDG,CPI,UEM,Rel GDP,Rel GDC,Annual Volatility
ZAF2000,2000,4.200003,5.338952,29.879999,0.018783,0.078699,0.186650
NOR2000,2000,3.204979,3.085506,3.460000,0.023588,0.989604,0.090269
IND2000,2000,3.840991,4.009439,2.730000,0.064518,0.011505,0.040950
SWE2000,2000,4.766349,0.899144,5.470000,0.036204,0.768838,0.115088
JPN2000,2000,2.779633,-0.676579,4.700000,0.673222,1.000000,0.092698


In [ ]:
# @TODO: Linear regression